In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import SGD
import pandas as pd

import os
import tempfile
import math

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras import regularizers

data = pd.read_csv("UNSW_NB15_training-set.csv")
data_test = pd.read_csv("UNSW_NB15_testing-set.csv")

X_train = data
X_train.pop("id")
y_train = X_train.pop("label")

X_test = data_test
X_test.pop("id")
y_test = X_test.pop("label")

In [ ]:
def handle_non_numerical_data(df):
    columns = df.columns.values
    for column in columns:
        text_digit_vals = {}
        def convert_to_int(val):
            return text_digit_vals[val]

        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            column_contents = df[column].values.tolist()
            unique_elements = set(column_contents)
            x = 0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    text_digit_vals[unique] = x
                    x+=1

            df[column] = list(map(convert_to_int, df[column]))

    return df

X_train = handle_non_numerical_data(X_train)
X_test = handle_non_numerical_data(X_test)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)


# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [ ]:
epochs = 10
batch_size = 32

class_weights = class_weight.compute_class_weight(class_weight="balanced", classes=np.unique(y_train), y=y_train)
class_weights = dict(zip(np.unique(y_train), class_weights))

# Set initial learning rate and lr decay function
initial_lr = 0.01
def lr_step_decay(epoch, lr):
    epochs_per_drop = 5
    return initial_lr * math.pow(0.9, math.floor(epoch/epochs_per_drop))

model = tf.keras.Sequential([
  layers.Dense(44, activation='relu'),
  layers.Dense(44, activation='relu'),
  layers.Dense(12, activation='relu'),
  layers.Dense(12, activation='relu'),
  layers.Dense(1),
  layers.Activation("sigmoid")
])

# Compile model
model.compile(
    optimizer=SGD(learning_rate=0.01, momentum=0.9),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

# Train model with proper callback
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), batch_size=batch_size, epochs=epochs, class_weight=class_weights, verbose=1, callbacks=[LearningRateScheduler(lr_step_decay, verbose=1)])

print("\nTest data loss/accuracy:")
model.evaluate(X_test, y_test)


Epoch 1: LearningRateScheduler setting learning rate to 0.01.
Epoch 1/10
4384/4384 [==============================] - 15s 3ms/step - loss: 0.0450 - accuracy: 0.9833 - val_loss: 0.0315 - val_accuracy: 0.9906 - lr: 0.0100

Epoch 2: LearningRateScheduler setting learning rate to 0.01.
Epoch 2/10
4384/4384 [==============================] - 12s 3ms/step - loss: 0.0171 - accuracy: 0.9943 - val_loss: 0.0071 - val_accuracy: 0.9975 - lr: 0.0100

Epoch 3: LearningRateScheduler setting learning rate to 0.01.
Epoch 3/10
4384/4384 [==============================] - 11s 3ms/step - loss: 0.0105 - accuracy: 0.9961 - val_loss: 0.0032 - val_accuracy: 0.9991 - lr: 0.0100

Epoch 4: LearningRateScheduler setting learning rate to 0.01.
Epoch 4/10
4384/4384 [==============================] - 13s 3ms/step - loss: 0.0027 - accuracy: 0.9992 - val_loss: 0.0018 - val_accuracy: 0.9995 - lr: 0.0100

Epoch 5: LearningRateScheduler setting learning rate to 0.01.
Epoch 5/10
4384/4384 [==============================]

[0.31849250197410583, 0.9574041962623596]